In [1]:
# Importando as Bibliotecas
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

In [2]:
# Lendo os arquivos
csv_populacao_tratada = pd.read_csv('dataframe_populacao_tratada.csv')
csv_portfolio1 = pd.read_csv('portfolio1.csv')
csv_portfolio2 = pd.read_csv('portfolio2.csv')
csv_portfolio3 = pd.read_csv('portfolio3.csv')

# Criando DataFrame para cada um dos arquivos
df_populacao = pd.DataFrame(csv_populacao_tratada)
df_portfolio1 = pd.DataFrame(csv_portfolio1)
df_portfolio2 = pd.DataFrame(csv_portfolio2)
df_portfolio3 = pd.DataFrame(csv_portfolio3)

In [3]:
df_populacao.shape

(462298, 267)

In [4]:
df_portfolio1.shape

(555, 182)

In [5]:
df_portfolio2.shape

(566, 2)

In [6]:
df_portfolio3.shape

(265, 2)

## TREINO e TESTE

In [7]:
# DataFrame (df_entrada): é o DataFrame criado a partir da lista de clientes do usuário (upload arquivo)
# Queremos apenas o valor de "id". As outras colunas "não interessam, pois já temos as informações na população
df_entrada = pd.DataFrame(df_portfolio2['id'])

# Criando a coluna "target" no DataFrame que recebemos de entrada (que contém a lista das empresas (portfolio))
# E atribuindo o valor target = 1
df_entrada['target'] = 1

In [8]:
df_entrada.columns

Index(['id', 'target'], dtype='object')

In [9]:
df_completo = pd.merge(df_populacao, df_entrada, how='outer', on ='id')

In [10]:
df_completo.shape

(462298, 268)

In [11]:
df_completo.drop(columns = 'Unnamed: 0', inplace = True)

In [12]:
df_completo.shape

(462298, 267)

In [13]:
# Colocando zeros onde "Target" é NaN (ou seja, != 1)
df_completo['target'].fillna(value = 0, inplace=True)

In [14]:
df_completo['target'].isna().sum()

0

In [15]:
# Vamos trabalhar com o DataFrame (df_completo)

## RFE !

In [16]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [17]:
df_entrada.shape

(566, 2)

In [18]:
#df_completo_RFE = df_completo
#df_completo_RFE.drop(columns= 'id', inplace = True)

In [19]:
df_completo_RFE = pd.merge(df_entrada, df_populacao, how='inner', on ='id')

In [20]:
salvando_ID = df_completo_RFE['id']

In [21]:
df_ID = pd.DataFrame({'id': salvando_ID})
df_ID.head()

,id
0,09e95c1a84049001d086470a8f320a19b076f955a89122...
1,dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728...
2,16843c9ffb920170477118933798510d8d5f00f5c56c12...
3,ff045934d90aab184dd15e66daf3b3c97193bda5449298...
4,a0ce528caab2c62497a548d0ebd9e935143ef81ed7dbf0...


In [22]:
df_completo_RFE.drop(columns = 'id', inplace = True)

In [23]:
df_completo_RFE.shape

(566, 267)

In [24]:
Y_treino_RFE = df_completo_RFE['target']
X_treino_RFE = df_completo_RFE.drop(columns = 'target')

In [25]:
#reg= LinearRegression()
#reg.fit(X_train, y_train)
#rfe = RFE(reg)
#rfe.fit(X_train, y_train)

In [26]:
estimator = SVR(kernel="linear")

In [27]:
selector = RFE(estimator)

In [28]:
selecionadas = selector.fit(X_treino_RFE, Y_treino_RFE)

In [29]:
selecionadas

RFE(estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                  gamma='scale', kernel='linear', max_iter=-1, shrinking=True,
                  tol=0.001, verbose=False),
    n_features_to_select=None, step=1, verbose=0)

In [30]:
df_colunas_RFE_auxiliar = pd.DataFrame({'coluna': X_treino_RFE.columns, 'bool': selector.get_support()})
lista_colunas_NAO_selecionadas = list(df_colunas_RFE_auxiliar[df_colunas_RFE_auxiliar['bool'] == False]['coluna'])

In [31]:
len(lista_colunas_NAO_selecionadas)

133

### Preparando o dataFrame para usarmos!

In [32]:
df_completo_AUX = df_completo

In [33]:
df_completo_AUX.shape

(462298, 267)

In [34]:
lista_colunas_NAO_selecionadas.remove('Unnamed: 0')

In [35]:
df_colunasSelecionadas = df_completo_AUX.drop(columns = lista_colunas_NAO_selecionadas)

In [36]:
df_colunasSelecionadas.shape

(462298, 135)

In [37]:
df_colunasSelecionadas.columns

Index(['id', 'nm_divisao_AGRICULTURA PECUARIA E SERVICOS RELACIONADOS',
       'nm_divisao_ALIMENTACAO', 'nm_divisao_ALOJAMENTO',
       'nm_divisao_ALUGUEIS NAO IMOBILIARIOS E GESTAO DE ATIVOS INTANGIVEIS NAO FINANCEIROS',
       'nm_divisao_ARMAZENAMENTO E ATIVIDADES AUXILIARES DOS TRANSPORTES',
       'nm_divisao_ATIVIDADES ARTISTICAS CRIATIVAS E DE ESPETACULOS',
       'nm_divisao_ATIVIDADES AUXILIARES DOS SERVICOS FINANCEIROS SEGUROS PREVIDENCIA COMPLEMENTAR E PLANOS DE SAUDE',
       'nm_divisao_ATIVIDADES CINEMATOGRAFICAS PRODUCAO DE VIDEOS E DE PROGRAMAS DE TELEVISAO GRAVACAO DE SOM E EDICAO DE MUSICA',
       'nm_divisao_ATIVIDADES DE APOIO A EXTRACAO DE MINERAIS',
       ...
       'de_faixa_faturamento_estimado_grupo_DE R$ 10.000.000,01 A R$ 30.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 100.000.000,01 A R$ 300.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 30.000.000,01 A R$ 100.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R

### Avaliando: SVM.OneClass : TREINO e TESTE + SVM.OneClass
#### Vou juntar todas as partes aqui para conseguir avaliar o algoritmo com os portfolios passados!

#### TREINO e TESTE

In [38]:
df_completo = df_colunasSelecionadas

In [39]:
df_completo.shape

(462298, 135)

In [40]:
# Salvando os valores de "id" em (ArmazenandoID)
ArmazenandoID = df_completo['id']
#df_completo.drop(columns = 'id', inplace=True)

In [41]:
# Definindo a base de TREINO (portfolio): Y_treino, X_treino
# Salvando em "df_completo_TREINO" as linhas do DataFrame que tem "target = 1"
df_completo_TREINO = df_completo.loc[(df_completo['target'] == 1)]

Y_treino = df_completo_TREINO['target']
X_treino = df_completo_TREINO.drop(columns ='target')

In [42]:
X_treino_split, X_teste_split, Y_treino_split, Y_teste_split = train_test_split(X_treino, Y_treino, test_size=0.2)

In [43]:
X_treino_split.shape

(452, 134)

In [44]:
X_teste_split.shape

(114, 134)

In [45]:
Y_treino_split.shape

(452,)

In [46]:
Y_teste_split.shape

(114,)

In [47]:
Y_treino = Y_treino_split
X_treino = X_treino_split

In [48]:
lista_ID_X_TREINO = list(X_treino['id'])

In [49]:
df_lista_ID_X_TREINO = pd.DataFrame({'id': lista_ID_X_TREINO})

In [50]:
# Definindo a base de TEST: X_teste, Y_predito
# (população, excluindo as empresas que fazem parte da lista da empresa, ou seja, "target = 0")
# Salvando em "df_result_0" as linhas do DataFrame que tem "target = 0"
df_completo_TESTE = df_completo.loc[(df_completo['target'] == 0)]
X_teste = df_completo_TESTE.drop(columns ='target')
# Y_predito: vai ser o resultado da previsão do Modelo

In [51]:
X_teste_split.shape

(114, 134)

In [52]:
Y_teste_split.shape

(114,)

In [53]:
X_teste.columns

Index(['id', 'nm_divisao_AGRICULTURA PECUARIA E SERVICOS RELACIONADOS',
       'nm_divisao_ALIMENTACAO', 'nm_divisao_ALOJAMENTO',
       'nm_divisao_ALUGUEIS NAO IMOBILIARIOS E GESTAO DE ATIVOS INTANGIVEIS NAO FINANCEIROS',
       'nm_divisao_ARMAZENAMENTO E ATIVIDADES AUXILIARES DOS TRANSPORTES',
       'nm_divisao_ATIVIDADES ARTISTICAS CRIATIVAS E DE ESPETACULOS',
       'nm_divisao_ATIVIDADES AUXILIARES DOS SERVICOS FINANCEIROS SEGUROS PREVIDENCIA COMPLEMENTAR E PLANOS DE SAUDE',
       'nm_divisao_ATIVIDADES CINEMATOGRAFICAS PRODUCAO DE VIDEOS E DE PROGRAMAS DE TELEVISAO GRAVACAO DE SOM E EDICAO DE MUSICA',
       'nm_divisao_ATIVIDADES DE APOIO A EXTRACAO DE MINERAIS',
       ...
       'de_faixa_faturamento_estimado_grupo_DE R$ 1.500.000,01 A R$ 4.800.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 10.000.000,01 A R$ 30.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 100.000.000,01 A R$ 300.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 3

In [54]:
lista_X_ID_TESTE_split = list(X_teste_split['id'])
lista_X_ID_TREINO_split = list(X_treino_split['id'])

In [55]:
df_X_ID_TESTE_split = pd.DataFrame({'id': lista_X_ID_TESTE_split})
df_X_ID_TREINO_split = pd.DataFrame({'id': lista_X_ID_TREINO_split})

In [56]:
df_X_ID_TREINO_split.shape

(452, 1)

In [57]:
df_X_ID_TESTE_split.head()

,id
0,2e133e2783fa227cc615aef93f4420f6d814f0b3bd1e1b...
1,50db4ee673eb3137a5cfb79a732a3f45a8d15f9f42e6b1...
2,f9657d3dca78c3d1a6f26f22379ef83aee9d4ecec19699...
3,3a2258e745f338ee08c5ef5ae655485273570afadfb467...
4,10091057bcba7164f6b45f10adc2bcc8f7a847bbabc09c...


In [58]:
X_teste.shape

(461732, 134)

In [59]:
# (X_teste_split): DataFrame teste dos treino COM 20% do portfolio
# (X_teste) com (df_X_teste_split)
Frames = (X_teste, X_teste_split)
X_teste_final = pd.concat(Frames)

In [60]:
X_teste_final

,id,nm_divisao_AGRICULTURA PECUARIA E SERVICOS RELACIONADOS,nm_divisao_ALIMENTACAO,nm_divisao_ALOJAMENTO,nm_divisao_ALUGUEIS NAO IMOBILIARIOS E GESTAO DE ATIVOS INTANGIVEIS NAO FINANCEIROS,nm_divisao_ARMAZENAMENTO E ATIVIDADES AUXILIARES DOS TRANSPORTES,nm_divisao_ATIVIDADES ARTISTICAS CRIATIVAS E DE ESPETACULOS,nm_divisao_ATIVIDADES AUXILIARES DOS SERVICOS FINANCEIROS SEGUROS PREVIDENCIA COMPLEMENTAR E PLANOS DE SAUDE,nm_divisao_ATIVIDADES CINEMATOGRAFICAS PRODUCAO DE VIDEOS E DE PROGRAMAS DE TELEVISAO GRAVACAO DE SOM E EDICAO DE MUSICA,nm_divisao_ATIVIDADES DE APOIO A EXTRACAO DE MINERAIS,...,"de_faixa_faturamento_estimado_grupo_DE R$ 1.500.000,01 A R$ 4.800.000,00","de_faixa_faturamento_estimado_grupo_DE R$ 10.000.000,01 A R$ 30.000.000,00","de_faixa_faturamento_estimado_grupo_DE R$ 100.000.000,01 A R$ 300.000.000,00","de_faixa_faturamento_estimado_grupo_DE R$ 30.000.000,01 A R$ 100.000.000,00","de_faixa_faturamento_estimado_grupo_DE R$ 300.000.000,01 A R$ 500.000.000,00","de_faixa_faturamento_estimado_grupo_DE R$ 360.000,01 A R$ 1.500.000,00","de_faixa_faturamento_estimado_grupo_DE R$ 4.800.000,01 A R$ 10.000.000,00","de_faixa_faturamento_estimado_grupo_DE R$ 500.000.000,01 A 1 BILHAO DE REAIS","de_faixa_faturamento_estimado_grupo_DE R$ 81.000,01 A R$ 360.000,00",fl_optante_simples
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173799,17cf214c3114d1ebcc7b07f603ceda9e100788c0573555...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
67168,87f6d7989c1ee4c59f3db38536222d5b3d909f65448313...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
214552,0fab8c3fe5f91e5a495740c6129c5e9af830d1e0790162...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
408455,75c5736fc78a378ceebcbd125927004f8b442f62425dfc...,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [61]:
lista_ID_X_teste_Final = list(X_teste_final['id'])

In [62]:
# DataFrame com uma única coluna "id" com os ids de todos as colunas em "TREINO"
df_lista_ID_X_teste_Final = pd.DataFrame({'id': lista_ID_X_teste_Final})

In [63]:
df_lista_ID_X_teste_Final.columns

Index(['id'], dtype='object')

In [64]:
X_teste_final.shape

(461846, 134)

In [65]:
# Lembrando que os "ids" estão armazenados em (lista_ID_X_teste_Final)/(df_lista_ID_X_teste_Final)
X_teste_final.drop(columns = 'id', inplace =True)

In [66]:
X_teste_final.shape

(461846, 133)

In [67]:
# Lembrando que os "ids"estão armazenados em (lista_ID_X_TREINO)/(df_lista_ID_X_TREINO)
X_treino.drop(columns = 'id', inplace =True)

In [68]:
X_treino.shape

(452, 133)

In [69]:
X_teste_final.isna().sum()

nm_divisao_AGRICULTURA PECUARIA E SERVICOS RELACIONADOS                                0
nm_divisao_ALIMENTACAO                                                                 0
nm_divisao_ALOJAMENTO                                                                  0
nm_divisao_ALUGUEIS NAO IMOBILIARIOS E GESTAO DE ATIVOS INTANGIVEIS NAO FINANCEIROS    0
nm_divisao_ARMAZENAMENTO E ATIVIDADES AUXILIARES DOS TRANSPORTES                       0
                                                                                      ..
de_faixa_faturamento_estimado_grupo_DE R$ 360.000,01 A R$ 1.500.000,00                 0
de_faixa_faturamento_estimado_grupo_DE R$ 4.800.000,01 A R$ 10.000.000,00              0
de_faixa_faturamento_estimado_grupo_DE R$ 500.000.000,01 A 1 BILHAO DE REAIS           0
de_faixa_faturamento_estimado_grupo_DE R$ 81.000,01 A R$ 360.000,00                    0
fl_optante_simples                                                                     0
Length: 133, dtype: i

#### OneClass.SVM

In [70]:
# OneClass.SVM
one_class_SVM = OneClassSVM(kernel='rbf')
resultado_OneClass_FIT = one_class_SVM.fit(X_treino)
resultado_OneClass_PREDICT = one_class_SVM.predict(X_teste_final)

#### Analisando o Resultado!

In [71]:
# df_ArmazenandoID.head()

In [72]:
# Criando o DataFrame (df_resultado) e igualamos ao (df_ArmazenandoID)
# Falta: adicionar o (df_completo) ao (df_resultado)
# Falta: adicionar o resultado predito! (df_resultado_OneClass_PREDICT, DataFrame com uma única coluna)
df_resultado = pd.DataFrame(df_lista_ID_X_teste_Final)

In [73]:
# Juntando o DataFrame (df_resultado) com com DataFrame (df_completo)
# df_resultado = df_resultado.join(df_completo)

In [74]:
# Criando um DataFrame com uma única coluna que contém o resultado predito!
df_resultado_OneClass_PREDICT = pd.DataFrame({'resultado_OneClass_PREDICT': resultado_OneClass_PREDICT})

In [75]:
# Juntando o DataFrame (df_resultado) com com DataFrame (resultado_OneClass_PREDICT)
df_resultado = df_resultado.join(df_resultado_OneClass_PREDICT)

In [76]:
df_resultado.head()

,id,resultado_OneClass_PREDICT
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,-1
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,-1
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,-1
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,-1
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,-1


In [77]:
# Pronto, agora temos um DataFrame (df_resultado) com os valores de: ID, resultadoPredict
# Agora temos que avaliar qual a % dos valores do portfolio foram preditas!

# Lembrando que (lista_X_teste_split)/(df_X_teste_split):
# É a LISTA com 20% dos "ids" presente no portfolio de entrada que "se juntaram a população"
# É o DataFrame com 20% dos "ids" presente no portfolio de entrada que "se juntaram a população"

In [78]:
# O DataFrame (df_X_TESTE_split) contém os "ids" ...
# ... corresponde aos 20% dos "ids" presente no portfólio de entrada que estavam na população

# OBJETIVO: Calcular
# Como:
# SOMAR: quantidade de '1'em df_vamosAnalisar['resultado_OneClass_PREDICT']
# Calcular %: SomaDos1 / Tamanho total
#df_vamosAnalisar = df_resultado.loc[df_resultado['id'] == lista_ID_70_portfolioEntrada]

In [79]:
#df_X_TESTE_split

In [80]:
df_resultado.columns

Index(['id', 'resultado_OneClass_PREDICT'], dtype='object')

In [81]:
# Os resultados de (resultado_OneClass_PREDICT) são (-1 e 1)
df_resultado['resultado_OneClass_PREDICT'].unique()

array([-1,  1])

In [82]:
# Vamos substituir os valores -1 por 0
df_resultado['resultado_OneClass_PREDICT'].replace({-1: 0}, inplace = True)

In [83]:
# Testando para ver se mudamos os valores resultado de (-1 e 1) para (0 e 1)
# Resposta: Ficou ótimo :D
df_resultado['resultado_OneClass_PREDICT'].unique()

array([0, 1])

In [84]:
df_axu = pd.DataFrame()

In [85]:
valorTotal = 0
tamanho = len(lista_X_ID_TESTE_split)

lista = []
for x in lista_X_ID_TESTE_split:
    # print('x',x)
    valor = list(df_resultado[df_resultado['id'] == x]['resultado_OneClass_PREDICT'])
    valorTotal = valorTotal + valor[0]


In [86]:
# Verificando % dos que eu já sabia que fazia parte!
acerto = (valorTotal/tamanho)
print('O % de acerto foi: ', acerto*100)

O % de acerto foi:  46.49122807017544
